In [3]:
import pandas as pd
import mysql.connector

In [14]:
connection = mysql.connector.connect(
    host= 'localhost',         
    user= 'root',             
    password= '123456', 
    database = 'day_47'     
)
cursor = connection.cursor()

### Bài 1

In [5]:
cursor.execute("""
CREATE TABLE Category (
    CategoryID INTEGER PRIMARY KEY AUTO_INCREMENT,
    Name VARCHAR(200) NOT NULL
);
""")

In [6]:
cursor.execute("""
CREATE TABLE Product (
    ProductID INTEGER PRIMARY KEY AUTO_INCREMENT,
    Name VARCHAR(200) NOT NULL, 
    CategoryID INTEGER NOT NULL,
    Price DECIMAL(10, 0) NOT NULL, 
    FOREIGN KEY (CategoryID) REFERENCES Category(CategoryID)
);
""")

In [7]:
cursor.execute("""
CREATE TABLE Variant (
    VariantID INTEGER PRIMARY KEY AUTO_INCREMENT,
    ProductID INTEGER NOT NULL,
    Color VARCHAR(20) NOT NULL, 
    Size VARCHAR(5) NOT NULL, 
    Stock INT NOT NULL,
    FOREIGN KEY (ProductID) REFERENCES Product(ProductID)
);
""")

In [8]:
cursor.execute("""
CREATE TABLE Customer (
    CustomerID INTEGER PRIMARY KEY AUTO_INCREMENT,
    Name TEXT NOT NULL,
    Phone VARCHAR(15) UNIQUE, 
    Address TEXT NOT NULL, 
    CreatedDate DATE NOT NULL
);
""")

In [ ]:
cursor.execute("""
CREATE TABLE Orders (
    OrderID INTEGER PRIMARY KEY AUTO_INCREMENT,
    CustomerID INTEGER NOT NULL,
    OrderDate DATE NOT NULL,
    Status ENUM('Đã đặt', 'Đã giao') NOT NULL,
    FOREIGN KEY (CustomerID) REFERENCES Customer(CustomerID)
);
""")

In [13]:
cursor.execute("""
CREATE TABLE OrderDetail (
    OrderID INTEGER NOT NULL,
    VariantID INTEGER NOT NULL,
    Quantity INTEGER NOT NULL,
    Price DECIMAL(10, 0) NOT NULL, 
    FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
    FOREIGN KEY (VariantID) REFERENCES Variant(VariantID)
);
""")

### Bài 2

In [14]:
cursor.execute("""
INSERT INTO Category (CategoryID, Name) VALUES
(1, 'Áo'),
(2, 'Quần'),
(3, 'Váy'),
(4, 'Phụ kiện');
""")

connection.commit()

In [15]:
cursor.execute("""
INSERT INTO Product (ProductID, Name, CategoryID, Price) VALUES
(101, 'Áo sơ mi nam', 1, 300000),
(102, 'Quần jeans', 2, 500000),
(103, 'Váy xòe', 3, 450000);
""")

connection.commit()

In [17]:
cursor.execute("""
INSERT INTO Variant (VariantID, ProductID, Color, Size, Stock) VALUES
(1, 101, 'Trắng', 'M', 20),
(2, 101, 'Trắng', 'L', 15),
(3, 102, 'Xanh', 'M', 10),
(4, 103, 'Đỏ', 'S', 5);
""")

connection.commit()

In [18]:
cursor.execute("""
INSERT INTO Customer (CustomerID, Name, Phone, Address, CreatedDate) VALUES
(1, 'Trần An', '0909123456', 'Hà Nội', '2024-01-10'),
(2, 'Lê Bình', '0909223456', 'TP.HCM', '2024-02-05');
""")

connection.commit()

In [23]:
cursor.execute("""
INSERT INTO Orders (OrderID, CustomerID, OrderDate, Status) VALUES
(1, 1, '2024-03-01', 'Đã giao'),
(2, 2, '2024-03-02', 'Đã đặt');
""")

connection.commit()

In [24]:
cursor.execute("""
INSERT INTO OrderDetail (OrderID, VariantID, Quantity, Price) VALUES
(1, 1, 2, 300000),
(1, 3, 1, 500000),
(2, 4, 1, 450000);
""")

connection.commit()

### Bài 3

In [5]:
query = ("""
SELECT a.OrderID
    , b.Name
    , c.Total_Price
 FROM Orders a
LEFT JOIN Customer b on a.CustomerID = b.CustomerID
LEFT JOIN (SELECT OrderID, SUM(Price) Total_Price FROM OrderDetail GROUP BY OrderID) c on a.OrderID = c.OrderID;
""")

data_3 = pd.read_sql_query(query, connection)
print(data_3)

   OrderID     Name  Total_Price
0        1  Trần An     800000.0
1        2  Lê Bình     450000.0


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_18140\1212662664.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_3 = pd.read_sql_query(query, connection)


### Bài 4

In [16]:
query = ("""
SELECT a.ProductID
    , a.Name
    , b.Total_Stock
 FROM Product a
RIGHT JOIN (SELECT ProductID, Total_Stock FROM (SELECT ProductID
         , SUM(Stock) Total_Stock 
         FROM Variant 
         GROUP BY ProductID) a WHERE Total_Stock < 10) b on a.ProductID = b.ProductID;
""")

data_4 = pd.read_sql_query(query, connection)
print(data_4)

   ProductID     Name  Total_Stock
0        103  Váy xòe          5.0


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_18140\553456145.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_4 = pd.read_sql_query(query, connection)


### Bài 5

In [17]:
query = ("""
SELECT a.OrderDate
    , b.Name
    , c.Total_Price
 FROM Orders a
LEFT JOIN Customer b on a.CustomerID = b.CustomerID
LEFT JOIN (SELECT OrderID, SUM(Price) Total_Price FROM OrderDetail GROUP BY OrderID) c on a.OrderID = c.OrderID;
""")

data_5 = pd.read_sql_query(query, connection)
print(data_5)

    OrderDate     Name  Total_Price
0  2024-03-01  Trần An     800000.0
1  2024-03-02  Lê Bình     450000.0


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_18140\2845795595.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_5 = pd.read_sql_query(query, connection)


### Bài 6

In [18]:
query = ("""
SELECT a.CustomerID
    , a.Name
 FROM Customer a
WHERE 1 = 1 
AND a.CustomerID IN (SELECT CustomerID FROM Orders);
""")

data_6 = pd.read_sql_query(query, connection)
print(data_6)

   CustomerID     Name
0           1  Trần An
1           2  Lê Bình


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_18140\2398750012.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_6 = pd.read_sql_query(query, connection)


### Bài 7

In [22]:
query = ("""
SELECT OrderID
    , SUM(Quantity) Total_Quantity
 FROM OrderDetail
GROUP BY OrderID 
ORDER BY Total_Quantity DESC
LIMIT 1;
""")

data_7 = pd.read_sql_query(query, connection)
print(data_7)

   OrderID  Total_Quantity
0        1             3.0


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_18140\838066937.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_7 = pd.read_sql_query(query, connection)


### Bài 8

In [23]:
query = ("""
SELECT c.Name
    , b.Color
    , SUM(a.Price) Total_Price
 FROM OrderDetail a
LEFT JOIN Variant b on a.VariantID = b.VariantID
LEFT JOIN Product c on b.ProductID = c.ProductID
GROUP BY c.Name, b.Color;
""")

data_8 = pd.read_sql_query(query, connection)
print(data_8)

           Name  Color  Total_Price
0  Áo sơ mi nam  Trắng     300000.0
1    Quần jeans   Xanh     500000.0
2       Váy xòe     Đỏ     450000.0


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_18140\2269107156.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_8 = pd.read_sql_query(query, connection)


### Bài 9

In [27]:
query = ("""
SELECT c.Name
    , b.Color
    , b.Size
    , SUM(a.Quantity) Total_Quantity
    , SUM(a.Price) Total_Price
 FROM OrderDetail a
LEFT JOIN Variant b on a.VariantID = b.VariantID
LEFT JOIN Product c on b.ProductID = c.ProductID
GROUP BY c.Name, b.Color, b.Size;
""")

data_9 = pd.read_sql_query(query, connection)
print(data_9)

           Name  Color Size  Total_Quantity  Total_Price
0  Áo sơ mi nam  Trắng    M             2.0     300000.0
1    Quần jeans   Xanh    M             1.0     500000.0
2       Váy xòe     Đỏ    S             1.0     450000.0


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_18140\3704359340.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_9 = pd.read_sql_query(query, connection)


### Bài 10

In [28]:
query = ("""
SELECT a.CustomerID
    , a.Name
 FROM Customer a
WHERE 1 = 1 
AND a.CustomerID NOT IN (SELECT CustomerID FROM Orders);
""")

data_10 = pd.read_sql_query(query, connection)
print(data_10)

Empty DataFrame
Columns: [CustomerID, Name]
Index: []


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_18140\729503367.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_10 = pd.read_sql_query(query, connection)


### Bài 11

In [33]:
query = ("""
SELECT MONTH(a.OrderDate) OrderMonth, b.Name, b.Phone, b.Address
FROM Orders a 
LEFT JOIN Customer b on a.CustomerID = b.CustomerID
WHERE a.CustomerID IN (SELECT CustomerID FROM Customer
WHERE CustomerID NOT IN (SELECT CustomerID FROM Orders));
""")

data_11 = pd.read_sql_query(query, connection)
print(data_11)

Empty DataFrame
Columns: [OrderMonth, Name, Phone, Address]
Index: []


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_18140\2914171370.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_10 = pd.read_sql_query(query, connection)


### Bài 12

In [34]:
query = ("""
SELECT a.Status, COUNT(a.OrderID) Total_Order
FROM Orders a 
GROUP BY a.Status;
""")

data_12 = pd.read_sql_query(query, connection)
print(data_12)

    Status  Total_Order
0  Đã giao            1
1   Đã đặt            1


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_18140\1077863239.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_12 = pd.read_sql_query(query, connection)


### Bài 13

In [35]:
query = ("""
SELECT a.ProductID
         , a.Name
         , b.Total_Stock
         , b.Total_Variant
         , b.Total_Stock / b.Total_Variant Average_Stock
FROM Product a 
LEFT JOIN (SELECT ProductID, SUM(Stock) Total_Stock, COUNT(VariantID) Total_Variant FROM Variant GROUP BY ProductID) b on a.ProductID = b.ProductID;
""")

data_13 = pd.read_sql_query(query, connection)
print(data_13)

   ProductID          Name  Total_Stock  Total_Variant  Average_Stock
0        101  Áo sơ mi nam         35.0              2           17.5
1        102    Quần jeans         10.0              1           10.0
2        103       Váy xòe          5.0              1            5.0


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_18140\1644216648.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_13 = pd.read_sql_query(query, connection)
